# here goes nothing!

following this tutorial: 
- Let's build GPT: from scratch, in code, spelled out.
- https://youtu.be/kCc8FmEb1nY?si=yQ_a5pSdg7JxcC9X

In [1]:
# get the dataset - he is using tiny shakespeare
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-05-24 05:16:27--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-05-24 05:16:28 (18.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# let's read it now
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [3]:
print("length of dataset in characters", len(text))

length of dataset in characters 1115394


In [4]:
# looking at the first 1000 characters -
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# what are the unique characters in this text?
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


we want to now "tokenize" - turn the characters into **sequence of integers**

In [6]:
# create map from characters to integers
  # turns out ch:i is labelling the key value pairs

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# encoder: function which takes a string, and for each character will look up in stoi. as an array of integers
encode = lambda s: [stoi[c] for c in s]
# decoder: function which for each integer in a list, look it up in itos, then join together as a string
decode = lambda l: ''.join(itos[i] for i in l)

print(encode("hii there"))
print(decode(encode("hii there")))

# but there are MANY ways -- eg. SentencePiece ->
# tiktoken -> this will encode words into pairs and this is what open ai uses. COOL!
# https://github.com/openai/tiktoken --

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


now let's try tokenizing all of tiny shakespeare!

In [7]:
# why are we using pytorch tensors?

import torch
data = torch.tensor(encode(text), dtype=torch.long) # tensor is most efficient numerical computation
print(data.shape, data.dtype) # this is how long, what type
print(data[:1000]) # first 1000 characters

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

in general practice, when we want to do machine learning...
# NEVER test on training data

hence one of the lessons learned in ML class is to split training and evaluation data

In [8]:
# thus let's split it now.... his code says to split exactly into 90/10
# would it make sense to randomly sample 90% and 10%? I guess in this case no because of LLMs?

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# this is what poe explains:
# In many machine learning tasks, random sampling is used to ensure a representative distribution of data in training and validation sets. This helps mitigate biases and ensures that the model is exposed to a variety of examples.
# In this case, not randomly sampling the data makes sense because of the sequential nature of text and the need to maintain contextual relationships for effective training of LLMs.

In [9]:
# we would generally train in *blocks*
# he calls them of lengths block_size:

block_size = 8
train_data[:block_size+1]

# this actually has multiple examples (to a transformer) - since transformer predicts at EACH spot
# for 9 pieces of data, there's 8 examples

# output is: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
# eg 1) 47 after 18
# eg 2) 56 after 47, 18,
# eg 3) 57, 56 .....
# this is *backprop function* in practice! (need to watch the formula, again)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
# just for illustration's sake --

x = train_data[:block_size] # inputs into the transformer
y = train_data[1:block_size+1] # outputs (next block, offset by one because we're looking
    # for the relationship btwn each character
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


the reason we do this, is so that transformer is used to seeing data of all
sizes between 1 to 8. after 8, we start "**TRUNCATING**", meaning we split it up

In [12]:
torch.manual_seed(1337) # setting random generator ONLY so that we see the same numbers
batch_size = 4 # we can run these in PARALLEL
block_size = 8 # max context length

def get_batch(split):
  # we're creating batches - training data flag we give training data
  data = train_data if split == 'train' else val_data

  # randomly get batch_size numbers, make sure we return "block_size" amts
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')

print(f"these are our inputs: {xb}")
print(f"these are out outputs: {yb}")

print('---')
# basically for each character, the next char depends on the previous array
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target is: {target}")

these are our inputs: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
these are out outputs: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
when input is [24] the target is: 43
when input is [24, 43] the target is: 58
when input is [24, 43, 58] the target is: 5
when input is [24, 43, 58, 5] the target is: 57
when input is [24, 43, 58, 5, 57] the target is: 1
when input is [24, 43, 58, 5, 57, 1] the target is: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
when input is [44] the target is: 53
when input is [44, 53] the target is: 56
when input is [44, 53, 56] the target is: 1
when input is [44, 53, 56, 1] the target is: 58
when input is [44, 53, 56, 1, 58] the target 

# let's train!

In [16]:
# firstly with: bigram language model - statistically based on previous word
# subclass of nn model (handles neural networks)

import torch
import torch.nn as nn # neural networks ahhh
from torch.nn import functional as F
torch.manual_seed(1337) # again just for reproducability

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__() # just inheriting from neural networks
    # each token directly reads off logits (nums for fn) for next token
    # based on a lookup table

    # what is nn.Embedding
    # QUOTE:
      # nn.Embedding is – like a linear layer – a M x N matrix,
      # with M being the number of words and N being the size of each word vector.
    # it's a wrapper of a tensor of shape vbsize x vbsize

    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)


  # this is the FORWARD PASS: The forward pass is the process of passing
    # input data through the layers of a neural network to obtain an output.
  def forward(self, idx, targets):
    #### 1. idx and targets are both (B, T) tensor of integers

    # each int will refer to that table to get the logits
    # plucks out a row based on the index in that table

    # eg. 43 num -> will get 43rd row,
    logits = self.token_embedding_table(idx)

    # (B, T, C) - batch, time, channel tensor
    # in our example: batch = 4, time = eight, channel = vocab size (65)
    # arrange rows in B x T x C


    # logits are the scores for the NEXT value


    #### 2. now we need to add a loss function (that we want to minimize to train)
    # it's also log loss; btwn 0 and 1, we want to minimize to become more accurate
    # measures the quality
        # loss2 = F.cross_entropy(logits, targets)
    # BUT THIS IS NOT CORRECT FN, we need to reshape first to B, T, C due to the fn
    B, T, C = logits.shape
    logits = logits.view(B*T, C) # batch x time (4 * 8)
    targets = targets.view(B*T)
    loss = F.cross_entropy(logits, targets)

    return logits, loss


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape) # torch.Size([32, 65])
print(loss) # tensor(4.8786, grad_fn=<NllLossBackward0>)

# this is what we expect: -ln(1/65) = 4.17...
# because our loss is bigger, there is "entropy" and not great (?)


# but OVERALL, we can now evaluate the quality~

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
